### 1. 데이터 변환 메소드

- map()과 apply() 메소드
  - map()
    - 함수를 인자로 전달하고, 그 함수를 모든 원소에 적용한다.
  - apply()
    - DataFrame의 경우, 열마다 다른 type의 데이터가 들어 있을 수 있다.
      - 따라서, 열마다 다른 데이터 처리가 필요한 경우가 일반적이다.
      - 이를 위해 제공되는 메소드이다.
    - 함수를 인자로 전달하고, 그 함수를 DataFrame의 모든 열에 적용한다.
  - (참고) applymap()
    - DataFrame의 applymap() 메소드가 있는데, 이는 최근 pandas에서는 map()으로 통합

#### (1) Map

- strip()으로 처리가 필요한 문자열을 갖고 있는 시리즈

In [114]:
import pandas as pd

series = pd.Series(['a b c\n', ' a b ', '     a', 'd    '])
series.values

array(['a b c\n', ' a b ', '     a', 'd    '], dtype=object)

- 시리즈의 .str 속성을 이용하여 모든 원소에 strip()을 적용

In [116]:
series.str.strip()

0    a b c
1      a b
2        a
3        d
dtype: object

- map() 메소드로 작성한다면, 다음과 같이 lambda function을 사용하여 strip()을 호출

In [118]:
series.map(lambda x : x.strip())

0    a b c
1      a b
2        a
3        d
dtype: object

- 일반 함수를 작성해서 함수 이름을 map의 인자로 전달
  - 좀 더 복잡한 구현이 필요할때, lambda function으로는 구현이 어렵기 때문에 사용함

In [120]:
def simple_func(x):
    print(x)
    y = x.strip()
    return y
    
series.map(simple_func)

a b c

 a b 
     a
d    


0    a b c
1      a b
2        a
3        d
dtype: object

- 문자열을 strip()하고, split()하여 리스트로 변환해보자

In [130]:
series = pd.Series(['a b c\n', ' a b ', '     a', 'd    '])
series

0    a b c\n
1       a b 
2          a
3      d    
dtype: object

- .str 속성을 이용하여 변환하자.
  - .str 속성은 문자열 메소드를 한번만 적용할 수 있기 때문에,
  - 우선 strip()을 적용하여 불필요한 개행문자, 스페이스를 제거하고,
  - 그 결과에 다시 split() 시켜야 한다.

In [128]:
series = series.str.strip()
series = series.str.split()
series = series.str[-1]
series

0    c
1    b
2    a
3    d
dtype: object

In [132]:
series.str.strip().split()

AttributeError: 'Series' object has no attribute 'split'

- map() 메소드를 사용하여, strip()과 split()을 같이 적용할 수 있다.

In [134]:
series = pd.Series(['a b c\n', ' a b ', '     a', 'd    '])
series = series.map(lambda x : x.strip().split())
series

0    [a, b, c]
1       [a, b]
2          [a]
3          [d]
dtype: object

## (2) apply()

- 다음과 같은 DataFrame의 '주소'열과 '행정구역'열에 strip()을 적용해보자.

In [136]:
df = pd.DataFrame({'주소': pd.Series(['a b c\n', ' a b ', '     a', 'd    ']),
                    '행정구역': pd.Series(['x ', 'y ', ' w', ' z'])})
print(df)

        주소 행정구역
0  a b c\n   x 
1     a b    y 
2        a    w
3    d        z


- map()을 사용하여 모든 문자열 데이터에 strip()을 적용한다.

In [138]:
df = df.map(lambda x : x.strip())
print(df)

      주소 행정구역
0  a b c    x
1    a b    y
2      a    w
3      d    z


- 다음과 같은 열마다 다른 dtype으로 구성된 DataFrame을 생각해보자.

In [140]:
df = pd.DataFrame({'주소': pd.Series(['a b c\n', ' a b ', '     a', 'd    ']),
                    '행정구역': pd.Series(['x ', 'y ', ' w', ' z']),
                    '인구수': pd.Series([10, 20, 15, 25])})
df

,주소,행정구역,인구수
0,a b c\n,x,10
1,a b,y,20
2,a,w,15
3,d,z,25


In [142]:
df.dtypes

주소      object
행정구역    object
인구수      int64
dtype: object

- 여기에 map()을 사용하게 된다면, '인구수' 열의 원소에 strip()을 호출할 수가 없을 것이다.

In [150]:
def func(x):
    print(x)
    return x.strip()
df.map(func)

a b c

 a b 
     a
d    
x 
y 
 w
 z
10


AttributeError: 'int' object has no attribute 'strip'

In [144]:
df.map(lambda x : x.strip())

AttributeError: 'int' object has no attribute 'strip'

- DataFrame은 열마다 다른 dtype을 갖는 것이 일반적이고, 이를 위해 apply() 메소드가 제공된다.

- 우선 apply() 메소드가 어떻게 동작하는지 살펴보자.

In [152]:
def func(x):
    print(x)
    return x

df.apply(func)

0    a b c\n
1       a b 
2          a
3      d    
Name: 주소, dtype: object
0    x 
1    y 
2     w
3     z
Name: 행정구역, dtype: object
0    10
1    20
2    15
3    25
Name: 인구수, dtype: int64


,주소,행정구역,인구수
0,a b c\n,x,10
1,a b,y,20
2,a,w,15
3,d,z,25


- 위 코드 결과를 보면, 각 열 series가 func(x)의 인자 x로 전달되고 있음을 알 수 있다.
- 그럼, func()을 열의 dtype별로 다르게 동작하도록 작성해보자.

In [156]:
def func(x):   # x는 Series
    if x.dtype != object:  # 전달 받은 열이 문자열이 아니라면 아무일도 하지 않고 x를 그대로 반환
        return x
    return x.str.strip()   # 전달 받은 열이 문자열이라면, strip()을 수행
    
df.apply(func).values

array([['a b c', 'x', 10],
       ['a b', 'y', 20],
       ['a', 'w', 15],
       ['d', 'z', 25]], dtype=object)

- x가 series이므로 x.str.strip()으로 작성된 부분을 map()을 사용하여 구현할 수도 있다. 

In [158]:
def func(x):
    if x.dtype != object:
        return x
    return x.map(lambda x : x.strip())
    
df = df.apply(func)
print(df)

      주소 행정구역  인구수
0  a b c    x   10
1    a b    y   20
2      a    w   15
3      d    z   25
